<a href="https://colab.research.google.com/github/ksb25395/CAP6779/blob/master/Semeval_Embedding_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
from gensim.models import KeyedVectors
EMBEDDING_FILE = '/content/gdrive/My Drive/Colab Notebooks/datasets/Google/GoogleNews-vectors-negative300.bin.gz'
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
import pandas as pd 
df=pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/datasets/Semeval-2017-4A-English/SemEval2017-task4-dev.subtask-A.english.INPUT.txt', sep="\t", header=None)

In [0]:
df.head()
df = df.drop(columns=3)

In [0]:
df.columns = ['id', 'sentiment', 'text']

In [0]:
df

,id,sentiment,tweet
0,619950566786113536,neutral,"Picturehouse's, Pink Floyd's, 'Roger Waters: T..."
1,619969366986235905,neutral,Order Go Set a Watchman in store or through ou...
2,619971047195045888,negative,If these runway renovations at the airport pre...
3,619974445185302528,neutral,If you could ask an onstage interview question...
4,619987808317407232,positive,A portion of book sales from our Harper Lee/Go...
...,...,...,...
20627,681877834982232064,neutral,@ShaquilleHoNeal from what I think you're aski...
20628,681879579129200640,positive,"Iran ranks 1st in liver surgeries, Allah bless..."
20629,681883903259357184,neutral,Hours before he arrived in Saudi Arabia on Tue...
20630,681904976860327936,negative,@VanityFair Alex Kim Kardashian worth how to ...


In [0]:
df.to_csv('/content/gdrive/My Drive/Colab Notebooks/datasets/Semeval-2017-4A-English/SemEval2017-task4-dev.subtask-A.english.INPUT.csv', sep="\t", header=None)

In [0]:
import numpy as np
import re
import nltk
import spacy
import string

In [0]:
# Lower casing
df["text"] = df["text"].str.lower()
df.head()

,id,sentiment,tweet
0,619950566786113536,neutral,"picturehouse's, pink floyd's, 'roger waters: t..."
1,619969366986235905,neutral,order go set a watchman in store or through ou...
2,619971047195045888,negative,if these runway renovations at the airport pre...
3,619974445185302528,neutral,if you could ask an onstage interview question...
4,619987808317407232,positive,a portion of book sales from our harper lee/go...


In [0]:
# Removal of URLs
def remove_urls(text):
  url_pattern = re.compile(r'https?://\S+|www\.\S+')
  return url_pattern.sub(r'', text)
df["text"] = df["text"].apply(lambda x : remove_urls(x))
df.head()

# Remove @ mentions
def remove_mentions(text):
  text = re.sub(r"(?:\@|https?\://)\S+", "", text)
  return text
df["text"] = df["text"].apply(lambda x: remove_mentions(x))

def strip_links(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    return text
    
df["text"] = df["text"].apply(lambda x: strip_links(x))

def strip_all_entities(text):
    entity_prefixes = ['@','#']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

df["text"] = df["text"].apply(lambda x: strip_all_entities(x))

@Hrithink @abhishek t.co/5489dug89dgs98


In [0]:
# Conversion of emoticons to words
!pip install emot
from emot.emo_unicode import UNICODE_EMO, EMOTICONS
def convert_emoticons(text):
  for emot in EMOTICONS:
    re.sub(u'('+emot+')', "_".join(EMOTICONS[emot].replace(",","").split()), text)
  return text

df["text"] = df["text"].apply(lambda x : convert_emoticons(x))
df.head()

,id,sentiment,tweet
0,619950566786113536,neutral,picturehouse s pink floyd s roger waters the w...
1,619969366986235905,neutral,order go set a watchman in store or through ou...
2,619971047195045888,negative,if these runway renovations at the airport pre...
3,619974445185302528,neutral,if you could ask an onstage interview question...
4,619987808317407232,positive,a portion of book sales from our harper lee go...


In [0]:
# convert emoji to word
!pip install emoji
import emoji
def convert_emojis(text):
    return emoji.demojize(text)

df["text"] = df["text"].apply(lambda x : convert_emojis(x))
df.head()

     |████████████████████████████████| 51kB 2.5MB/s 
  Created wheel for emoji: filename=emoji-0.5.4-cp36-none-any.whl size=42175 sha256=9e0a4207d312522c5c9faf8d0fab97584b1f875c91842bcee81f508e5b5ee622
  Stored in directory: /root/.cache/pip/wheels/2a/a9/0a/4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji


,id,sentiment,tweet
0,619950566786113536,neutral,picturehouse s pink floyd s roger waters the w...
1,619969366986235905,neutral,order go set a watchman in store or through ou...
2,619971047195045888,negative,if these runway renovations at the airport pre...
3,619974445185302528,neutral,if you could ask an onstage interview question...
4,619987808317407232,positive,a portion of book sales from our harper lee go...


In [0]:
# !pip install autocorrect
# from autocorrect import Speller
# spell = Speller(lang='en')
# def correct_spellings(text):
#   return spell(text)

# df["text"] = df["text"].apply(lambda x: correct_spellings(x))
# df.head()

In [0]:
# expand contractions
!pip install -q contractions
import contractions
def expand_contractions(text):
  return contractions.fix(text)

df["text"] = df["text"].apply(lambda x : expand_contractions(x))
df.head()

     |████████████████████████████████| 317kB 5.9MB/s 
     |████████████████████████████████| 245kB 27.5MB/s 


,id,sentiment,tweet
0,619950566786113536,neutral,picturehouse s pink floyd s roger waters the w...
1,619969366986235905,neutral,order go set a watchman in store or through ou...
2,619971047195045888,negative,if these runway renovations at the airport pre...
3,619974445185302528,neutral,if you could ask an onstage interview question...
4,619987808317407232,positive,a portion of book sales from our harper lee go...


In [0]:
PUNCT = string.punctuation
def remove_punctuation(text):
  return text.translate(str.maketrans('', '', PUNCT))

df["text"] = df["text"].apply(lambda x : remove_punctuation(x))
df.head()

,id,sentiment,tweet
0,619950566786113536,neutral,picturehouse s pink floyd s roger waters the w...
1,619969366986235905,neutral,order go set a watchman in store or through ou...
2,619971047195045888,negative,if these runway renovations at the airport pre...
3,619974445185302528,neutral,if you could ask an onstage interview question...
4,619987808317407232,positive,a portion of book sales from our harper lee go...


In [0]:
from nltk.corpus import stopwords
nltk.download('stopwords')
", ".join(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [0]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
  return " ".join([word for word in str(text).split() if word not in STOPWORDS])

df["text"] = df["text"].apply(lambda x : remove_stopwords(x))
df.head()

,id,sentiment,tweet
0,619950566786113536,neutral,picturehouse pink floyd roger waters walll ope...
1,619969366986235905,neutral,order go set watchman store website tuesday ge...
2,619971047195045888,negative,runway renovations airport prevent seeing tayl...
3,619974445185302528,neutral,could ask onstage interview question miss usa ...
4,619987808317407232,positive,portion book sales harper lee go set watchman ...


In [0]:
from collections import Counter
cnt = Counter()
for text in df["text"].values:
  for word in text.split():
    cnt[word] += 1
cnt.most_common(10)

[('may', 3604),
 ('tomorrow', 2953),
 ('day', 1661),
 ('going', 1525),
 ('friday', 1312),
 ('see', 1303),
 ('amp', 1297),
 ('sunday', 1263),
 ('night', 1255),
 ('1st', 1245)]

In [0]:
FREQ_WORDS = set([word for (word, count) in cnt.most_common(10)])
def remove_freqwords(text):
  return " ".join([word for word in str(text).split() if word not in FREQ_WORDS])

df["text"] = df["text"].apply(lambda x : remove_freqwords(x))
df.head()

,id,sentiment,tweet
0,619950566786113536,neutral,picturehouse pink floyd roger waters walll ope...
1,619969366986235905,neutral,order go set watchman store website tuesday ge...
2,619971047195045888,negative,runway renovations airport prevent seeing tayl...
3,619974445185302528,neutral,could ask onstage interview question miss usa ...
4,619987808317407232,positive,portion book sales harper lee go set watchman ...


{('mbele', 1), ('lupa', 1), ('saatchi', 1), ('adventures', 1), ('dharma', 1), ('tai', 1), ('eyaaocampo', 1), ('nowtv', 1), ('oped', 1), ('anecdotes', 1), ('intellect', 1), ('skegness', 1), ('2chajnz', 1), ('vague', 1), ('loathe', 1), ('foranderson', 1), ('dolt', 1), ('ebbing', 1), ('mfw', 1), ('salafi', 1), ('cramp', 1), ('potty', 1), ('jazzstandardnyc', 1), ('notebook', 1), ('hoes', 1), ('bod', 1), ('welll', 1), ('cults', 1), ('enforcer', 1), ('handicapped', 1), ('qualifications', 1), ('everlasting', 1), ('fascinatingly', 1), ('nevill', 1), ('desks', 1), ('dgates', 1), ('karim', 1), ('dhana', 1), ('keepi', 1), ('sexist', 1), ('torture', 1), ('anot', 1), ('kanter', 1), ('crumbling', 1), ('emmanuel', 1), ('leathers', 1), ('aspe', 1), ('tbd', 1), ('excluding', 1), ('sudhar', 1), ('mcmichael', 1), ('exp', 1), ('broadeningday', 1), ('burt', 1), ('claude', 1), ('kendricks', 1), ('innovative', 1), ('templr', 1), ('wogan', 1), ('intoxicating', 1), ('coulson', 1), ('magician', 1), ('f206', 1),

In [0]:
n_rare_words = 10
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
def remove_rarewords(text):
    """custom function to remove the rare words"""
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])

df["text"] = df["text"].apply(lambda text: remove_rarewords(text))
df.head()

,id,sentiment,tweet
0,619950566786113536,neutral,picturehouse pink floyd roger waters walll ope...
1,619969366986235905,neutral,order go set watchman store website tuesday ge...
2,619971047195045888,negative,runway renovations airport prevent seeing tayl...
3,619974445185302528,neutral,could ask onstage interview question miss usa ...
4,619987808317407232,positive,portion book sales harper lee go set watchman ...


In [0]:
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
def lemmatize_words(text):
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

df["text"] = df["text"].apply(lambda text: lemmatize_words(text))
df.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


,id,sentiment,tweet
0,619950566786113536,neutral,picturehouse pink floyd roger water walll open...
1,619969366986235905,neutral,order go set watchman store website tuesday ge...
2,619971047195045888,negative,runway renovation airport prevent see taylor s...
3,619974445185302528,neutral,could ask onstage interview question miss usa ...
4,619987808317407232,positive,portion book sale harper lee go set watchman r...


In [0]:
# SMS Spelling Expansion

In [0]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_idx, test_idx in split.split(df, df["sentiment"]):
    train_set = df.loc[train_idx]
    test_set = df.loc[test_idx]

print(train_set["sentiment"].value_counts())
print(test_set["sentiment"].value_counts())

neutral     8273
positive    5647
negative    2585
Name: sentiment, dtype: int64
neutral     2069
positive    1412
negative     646
Name: sentiment, dtype: int64


In [0]:
train_set["sentiment"] = train_set["sentiment"].astype('category')
train_set["sentiment"] = train_set["sentiment"].cat.codes
train_set = train_set[["text", "sentiment"]]
test_set["sentiment"] = test_set["sentiment"].astype('category')
test_set["sentiment"] = test_set["sentiment"].cat.codes
test_set = test_set[["text", "sentiment"]]
print(train_set)
print(test_set)

                                                   tweet  sentiment
11706                still confuse kanye west speech lol          0
6270   talk naruto mom become huge fan within past ye...          2
14225  bryant huge homer tie cub rookie mark kris bry...          1
80     thank miss usa stole word beauty inside outsid...          1
12537  ott take annual leave immerse within first jan...          2
...                                                  ...        ...
8386   fan boy scout john cena character work real te...          2
19076  star war o2 centre join u 10am storytelling pr...          1
17034  um sir concede daughter rise star rahul gandhi...          1
17556         black kendrick lamar might best ever heard          2
17401  rahul gandhi aka raul vinci need bag check mat...          0

[16505 rows x 2 columns]
                                                   tweet  sentiment
3524   apart 4th inning mastery david price best part...          2
9205   dragon quest he

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

Using TensorFlow backend.


In [0]:
train_texts = train_set["text"]
test_texts = test_set["text"]

In [0]:
NUM_WORDS=25000
tokenizer = Tokenizer(num_words=NUM_WORDS,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'',
                      lower=True)
tokenizer.fit_on_texts(train_texts)
sequences_train = tokenizer.texts_to_sequences(train_texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 16475 unique tokens.


In [0]:
X_train = pad_sequences(sequences_train)
y_train = to_categorical(list(train_set["sentiment"]))

In [0]:
import numpy as np
EMBED_DIM = 300
vocab_size = min(NUM_WORDS, len(word_index) + 1)
embedding_matrix =np.random.normal(0,np.sqrt(0.25),[vocab_size, EMBED_DIM])

In [0]:
count = []
for word, i in word_index.items():
    if i >= vocab_size:
        continue
    try:
        embedding_vec = word2vec[word]
        embedding_matrix[i] = embedding_vec
    except KeyError:
        count.append(word)
        embedding_matrix[i] = np.random.normal(0,np.sqrt(0.25),EMBED_DIM)

In [0]:
count
len(count)

4146

In [0]:
from keras.layers import Embedding
embedding_layer = Embedding(vocab_size,
                            EMBED_DIM,
                            weights=[embedding_matrix],
                            trainable=True)
seq_len = X_train.shape[1]
from keras.layers import Input
from keras.layers import Flatten
from keras.layers import Dense
from keras.models import Model
from keras.layers import LSTM
from keras.layers import Dropout
inputs = Input(shape=(seq_len,))
embeddings = embedding_layer(inputs)
# flattened = Flatten()(embeddings)
lstm = LSTM(128, dropout=0.5)(embeddings)
dense = Dense(16, activation="relu")(lstm)
dropout = Dropout(rate=0.5)(dense)
output = Dense(y_train.shape[1], activation='softmax')(dropout)
model = Model(inputs, output)

In [0]:
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 29)                0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 29, 300)           4942800   
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               219648    
_________________________________________________________________
dense_5 (Dense)              (None, 16)                2064      
_________________________________________________________________
dropout_3 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 51        
Total params: 5,164,563
Trainable params: 5,164,563
Non-trainable params: 0
_________________________________________________

In [0]:
from keras.optimizers import Adam
adam = Adam(lr=1e-3)

model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['acc'])

In [0]:
model.fit(X_train, y_train, batch_size=1000, epochs=10, verbose=1, validation_split=0.1, shuffle=True)

Train on 14854 samples, validate on 1651 samples
Epoch 1/10
14854/14854 [==============================] - 19s 1ms/step - loss: 1.0140 - acc: 0.4956 - val_loss: 0.9738 - val_acc: 0.4924
Epoch 2/10
14854/14854 [==============================] - 17s 1ms/step - loss: 0.9593 - acc: 0.5224 - val_loss: 0.9092 - val_acc: 0.5603
Epoch 3/10
14854/14854 [==============================] - 17s 1ms/step - loss: 0.9124 - acc: 0.5316 - val_loss: 0.8460 - val_acc: 0.5954
Epoch 4/10
14854/14854 [==============================] - 17s 1ms/step - loss: 0.8513 - acc: 0.5515 - val_loss: 0.8131 - val_acc: 0.5960
Epoch 5/10
14854/14854 [==============================] - 17s 1ms/step - loss: 0.8088 - acc: 0.6027 - val_loss: 0.7925 - val_acc: 0.6451
Epoch 6/10
14854/14854 [==============================] - 17s 1ms/step - loss: 0.7616 - acc: 0.6576 - val_loss: 0.7702 - val_acc: 0.6584
Epoch 7/10
14854/14854 [==============================] - 17s 1ms/step - loss: 0.7234 - acc: 0.6789 - val_loss: 0.7487 - val_acc:

In [0]:
sequences_test=tokenizer.texts_to_sequences(test_set.text)
X_test = pad_sequences(sequences_test,maxlen=X_train.shape[1])
y_test = to_categorical(list(test_set['sentiment']))

In [0]:
model.evaluate(X_test, y_test)

4127/4127 [==============================] - 2s 427us/step


[0.7906004483341044, 0.6539859463089974]

In [0]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
from sklearn.metrics import f1_score
f1_score(y_true, y_pred, average='macro')

0.6173096513832949

The F-Score for the above model is 0.6173

In [0]:
from sklearn.metrics import confusion_matrix
z = confusion_matrix(y_true, y_pred)
print(z)

[[ 287  287   72]
 [ 171 1489  409]
 [  36  453  923]]


In [0]:
y_random = np.random.randint(3, size=len(y_true))

In [0]:
f1_score(y_true, y_random, average='macro')

0.3157743268108266

Random prediction metric is 0.316. That is, if we choose either negative, neutral or positive randomly, our F-score turns out to be around 0.32